In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms

# Miscellaneous Libraries
import time

In [19]:
class tutorial_model(nn.Module):
    def __init__(self):
        """ Initialize all layers of model """
        super(tutorial_model, self).__init__()
    
        # Convolution Layer 1
        self.convolution1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)  
        # Convolution Layer 2
        self.convolution2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)    
        
        # Dropout Layer
        self.conv_dropout = nn.Dropout()                                                 
        
        # Fully Connected Layer 1
        self.fc1 = nn.Linear(in_features=320, out_features=50)  
        # Fully Connected Layer 2
        self.fc2 = nn.Linear(in_features=50, out_features=10)                            
        
        # Max Pooling Layer
        self.pool = nn.MaxPool2d(2, 2)                                                   
        
    def forward(self, x):
        """ Chain all layers together """
        # input_x -> Convolution 1 -> RELU -> Pooling -> x_1
        x = self.pool(F.relu(self.convolution1(x)))    
        # x_1 -> Dropout -> x_2
        x = self.conv_dropout(x)                
        # x_2 -> Convolution 2 -> RELU -> Pooling -> x_3
        x = self.pool(F.relu(self.convolution2(x)))    
        
        # Collapse 3D tensor to 1D
        x = x.view(-1, 320)                            
        
        # x_3 -> Fully Connected 1 -> x_4 
        x = self.fc1(x)
        # x_4 -> Fully Connected 2 -> y
        y = self.fc2(x)                                
        return y

In [20]:
# Number of epochs for training
num_epochs = 3           

# Batch Size for training/testing
batch_size = 64          

# Learning Rate for optimizer
learning_rate = 0.01     

# Momentum for optimizer
momentum = 0.5           

# Dimensions of MNIST
dim = 32

In [21]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

# Transformation for training data
transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

In [22]:
# Download training data
trainset = torchvision.datasets.MNIST(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.MNIST(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

In [23]:
# Initialize previously defined model
model = tutorial_model()                                               

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();

In [24]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        
        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/3, Step: 1/938, Loss: 2.3080992698669434, Accuracy: 6.25%
Epoch: 1/3, Step: 101/938, Loss: 0.32894638180732727, Accuracy: 89.0625%
Epoch: 1/3, Step: 201/938, Loss: 0.25430336594581604, Accuracy: 87.5%
Epoch: 1/3, Step: 301/938, Loss: 0.321227103471756, Accuracy: 92.1875%
Epoch: 1/3, Step: 401/938, Loss: 0.19655072689056396, Accuracy: 92.1875%
Epoch: 1/3, Step: 501/938, Loss: 0.2646969258785248, Accuracy: 92.1875%
Epoch: 1/3, Step: 601/938, Loss: 0.2029225081205368, Accuracy: 95.3125%
Epoch: 1/3, Step: 701/938, Loss: 0.19168375432491302, Accuracy: 95.3125%
Epoch: 1/3, Step: 801/938, Loss: 0.14010334014892578, Accuracy: 96.875%
Epoch: 1/3, Step: 901/938, Loss: 0.1848112791776657, Accuracy: 96.875%
Epoch: 2/3, Step: 1/938, Loss: 0.12352896481752396, Accuracy: 95.3125%
Epoch: 2/3, Step: 101/938, Loss: 0.15378493070602417, Accuracy: 95.3125%
Epoch: 2/3, Step: 201/938, Loss: 0.12256582081317902, Accuracy: 95.3125%
Epoch: 2/3, Step: 301/938, Loss: 0.06944827735424042, Accuracy: 96.87

In [25]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 96.66%
